In [23]:
!pip install -q kaggle


In [24]:
!mkdir -p ~/.kaggle


In [25]:
!cp kaggle.json ~/.kaggle/


In [26]:
!ls ~/.kaggle


kaggle.json


In [27]:
!kaggle datasets download -d deadskull7/fer2013

fer2013.zip: Skipping, found more recently modified local copy (use --force to force download)


In [28]:
!unzip -q fer2013.zip -d .


replace ./fer2013.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [29]:
!ls

fer2013.csv  fer2013.zip  fer.json  kaggle.json  sample_data


In [30]:
import sys, os
import pandas as pd
import numpy as np
print(pd.__version__)
print(np.__version__)



1.0.5
1.18.5


In [31]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [32]:
print(keras.__version__)

2.3.1


In [33]:
df=pd.read_csv('fer2013.csv')


In [34]:
X_train,train_y,X_test,test_y=[],[],[],[]


In [35]:
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [36]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48


In [37]:
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [38]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)


In [39]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)


In [40]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)


In [41]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))


model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))


model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Flatten())


model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))


In [42]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [43]:
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Train on 28709 samples, validate on 3589 samples
Epoch 1/30
28709/28709 [==============================] - 29s 1ms/step - loss: 1.7252 - accuracy: 0.2935 - val_loss: 1.5681 - val_accuracy: 0.3784
Epoch 2/30
28709/28709 [==============================] - 22s 759us/step - loss: 1.5029 - accuracy: 0.4093 - val_loss: 1.4052 - val_accuracy: 0.4648
Epoch 3/30
28709/28709 [==============================] - 22s 758us/step - loss: 1.3978 - accuracy: 0.4612 - val_loss: 1.3108 - val_accuracy: 0.4901
Epoch 4/30
28709/28709 [==============================] - 22s 757us/step - loss: 1.3282 - accuracy: 0.4865 - val_loss: 1.2772 - val_accuracy: 0.5183
Epoch 5/30
28709/28709 [==============================] - 22s 760us/step - loss: 1.2839 - accuracy: 0.5070 - val_loss: 1.2556 - val_accuracy: 0.5291
Epoch 6/30
28709/28709 [==============================] - 22s 764us/step - loss: 1.2460 - accuracy: 0.5198 - val_loss: 1.2317 - val_accuracy: 0.5325
Epoch 7/30
28709/28709 [==============================] - 2

In [44]:

fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")
